# Creating a Medical triage workflow with TigerGraph
<ol>


# Set Up


In [2]:
!python --version
!pip install pyTigerGraph

Python 2.7.16


# Exploring pyTigerGraph

In [4]:
!pip install pyTigerGraph

In [6]:
import pyTigerGraph as tg # Importing

# Create a Connection
conn = tg.TigerGraphConnection(host="https://healthnet.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True)

In [7]:
# Sets up connection with GSQL
print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
Edge Types:

Graphs:
Jobs:


JSON API version: v2
Syntax version: v2



In [13]:
# Create Edges (published and references) and Vertices (source, article and event)
print(conn.gsql('''
  CREATE VERTEX condition(PRIMARY_ID name STRING, desciption STRING, icd_code STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX symptom(PRIMARY_ID name STRING, priority INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX precaution(PRIMARY_ID id STRING, description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX specialist(PRIMARY_ID name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"

  CREATE DIRECTED EDGE has_symptom(FROM condition, TO symptom) WITH REVERSE_EDGE="reverse_has_symptom"
  CREATE DIRECTED EDGE has_precaution(FROM condition, TO precaution) WITH REVERSE_EDGE="reverse_has_precaution"
  CREATE DIRECTED EDGE refer_to(FROM condition, TO specialist) WITH REVERSE_EDGE="reverse_refer_to"

  ''', options=[]))

print(conn.gsql('''CREATE GRAPH healthnet(condition, symptom, precaution, specialist, has_symptom, has_precaution, refer_to)''', options=[])) # Create the Graph



Successfully created vertex types: [condition].
Successfully created vertex types: [symptom].
Successfully created vertex types: [precaution].
Successfully created vertex types: [specialist].
Successfully created edge types: [has_symptom].
Successfully created reverse edge types: [reverse_has_symptom].
Successfully created edge types: [has_precaution].
Successfully created reverse edge types: [reverse_has_precaution].
Successfully created edge types: [refer_to].
Successfully created reverse edge types: [reverse_refer_to].
The graph healthnet is created.


In [14]:
# Edit the Connect Credentials for Loading Data
conn.graphname = "healthnet"
conn.apiToken = conn.getToken(conn.createSecret())

Now we can import the CSV media rating into the graph.

###  What is a ICD-10 diagnosis code?
Used for medical claim reporting in all healthcare settings, ICD-10-CM is a standardized classification system of diagnosis codes that represent conditions and diseases, related health problems, abnormal findings, signs and symptoms, injuries, external causes of injuries and diseases, and social circumstances

Jobs for importing CSV into the graph


In [15]:
print(conn.gsql('''
CREATE LOADING JOB load_job_specialists_csv_1650482322027 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX specialist VALUES($0) USING SEPARATOR=",", HEADER="false", EOL="\n";
    }
    ''', options=[]))

print(conn.gsql('''
CREATE LOADING JOB load_job_Symptom_severity_csv_1650482478169 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX symptom VALUES($0, _) USING SEPARATOR=",", HEADER="false", EOL="\n";
    }
 ''', options=[]))



Successfully created loading jobs: [load_job_specialists_csv_1650482322027].
Successfully created loading jobs: [load_job_Symptom_severity_csv_1650482478169].


## Peek into Data Load


In [16]:
import pandas as pd

df = pd.read_csv('data/conditions.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/specialists.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/symptoms.csv')
print(df.head(0))
print('######')


FileNotFoundError: [Errno 2] No such file or directory: 'data/conditions.csv'

### Clear the Whole Graph
DANGER ZONE

In [ ]:
# conn.gsql('''
# USE GLOBAL
# DROP ALL
# ''')

### Load data

In [18]:
import json

symptoms_file = "data/symptoms.csv"
res = conn.uploadFile(symptoms_file, fileTag='MyDataSource', jobName='load_job_Symptom_severity_csv_1650482478169')
print(json.dumps(res, indent=2))

null
